<a href="https://colab.research.google.com/github/RubhiniAnand/1.Python/blob/main/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict

# Helper functions to load appliances and issues
def create_appliances():
    appliances_info = [
        {
            "name": "Dishwasher",
            "issues": {
                "Dishes not cleaned properly": "Check for clogs in the spray arms and clean the filter. Avoid overloading and use an appropriate detergent.",
                "Dishwasher not draining": "Inspect and clean the drain filter and hose. Ensure the garbage disposal (if connected) is clear.",
                "Water leaks": "Check the door seal for cracks or damage. Tighten hose connections and inspect for any blockages in the drain."
            },
            "maintenance_tips": [
                "Clean the filter and spray arms monthly to prevent clogs.",
                "Run an empty cycle with vinegar once a month to remove buildup.",
                "Check and clean the door gasket every few months."
            ]
        },
        {
            "name": "Oven and Stovetop",
            "issues": {
                "Uneven heating": "Check and clean the oven's heating element. Recalibrate the thermostat if needed.",
                "Burners not igniting": "Ensure burner caps are seated properly. Clean the igniter and burner holes with a brush."
            },
            "maintenance_tips": [
                "Clean the oven and stovetop surfaces after each use to prevent buildup.",
                "Inspect and replace oven seals every six months to prevent heat loss.",
                "Use the self-cleaning function periodically and avoid using harsh chemicals."
            ]
        },
        {
            "name": "Air Conditioner",
            "issues": {
                "Not cooling effectively": "Check for clogged air filters and clean or replace them. Ensure vents are clear and unit is properly sized for the room.",
                "Water leakage": "Clear the condensate drain line and check the drain pan for cracks or clogs."
            },
            "maintenance_tips": [
                "Clean or replace air filters every month during peak use.",
                "Keep the area around the unit free of debris.",
                "Schedule professional maintenance annually to check refrigerant levels and overall efficiency."
            ]
        },
        {
            "name": "Refrigerator",
            "issues": {
                "Not cooling": "Ensure the thermostat is set correctly. Check condenser coils and clean if necessary.",
                "Ice maker not working": "Check the water supply line and ensure it’s not frozen or blocked. Restart the ice maker if necessary."
            },
            "maintenance_tips": [
                "Clean condenser coils twice a year to maintain efficiency.",
                "Check door seals regularly to prevent warm air from entering.",
                "Keep the refrigerator at 37°F (3°C) and freezer at 0°F (-18°C) for optimal performance."
            ]
        },
        {
            "name": "Washing Machine",
            "issues": {
                "Not spinning": "Ensure the load is balanced. Check for obstructions in the drain hose and clean the pump filter.",
                "Not draining water": "Locate and clean the drain filter, as it might be clogged with lint, debris, or small items.",
                "Excessive vibration": "Make sure the washer is level. Distribute clothes evenly in the drum and avoid overloading."
            },
            "maintenance_tips": [
                "Clean the detergent drawer and rubber door seal monthly to prevent mold buildup.",
                "Run a hot water cycle with vinegar once a month to clean the drum.",
                "Inspect hoses for leaks every six months and replace them every five years."
            ]
        }
    ]
    with open('appliances.json', 'w') as file:
        json.dump(appliances_info, file, indent=4)
    return appliances_info

def get_appliances():
    with open('appliances.json', 'r') as file:
        appliances_info = json.load(file)
    return appliances_info

def get_products_and_issue():
    appliances_info = get_appliances()
    products_by_issue = defaultdict(list)

    for appliance in appliances_info:
        appliance_name = appliance["name"]
        appliance_issues = appliance["issues"]
        maintenance_tips = appliance.get("maintenance_tips", [])

        for issue, solution in appliance_issues.items():
            products_by_issue[issue].append({
                "appliance": appliance_name,
                "solution": solution,
                "maintenance_tips": maintenance_tips
            })
    return dict(products_by_issue)

# Use OpenAI to find the appropriate response
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message["content"]

def find_product_and_issue(user_input, products_and_issues):
    response_data = []
    for issue, appliances in products_and_issues.items():
        if issue.lower() in user_input.lower():
            for appliance in appliances:
                response_data.append({
                    "product name": appliance["appliance"],
                    "Issue": issue,
                    "Solution": appliance["solution"],
                    "Maintenance Tips": appliance["maintenance_tips"],
                    "Safety Suggestion": "For any issues involving wiring or internal components, consider contacting a technician for safety."

                })
    return response_data

def process_user_message(user_input, all_messages, debug=True):

    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"]:
        print("1.Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("1.Input passed moderation check.")

    products_and_issues = get_products_and_issue()
    product_and_issue_response = find_product_and_issue(user_input, products_and_issues)

    if not product_and_issue_response:
        return "I'm unable to find the requested appliance issue. Please check if you’ve specified the correct product or issue.", all_messages

    # Prepare response message content
    system_message = """
    You are a customer service assistant for Home Appliances Maintenance System. Respond with helpful, concise solutions based on the appliance and issue provided. Include safety tips and suggest consulting a technician for complex or risky repairs.
    You are a customer service assistant for Home Appliances Maintenance System. \
    Provide solutions and regular maintenance tips from the  for home appliances based on the issue. \
    Advise to contact a technician if the issue involves potential safety risks (e.g., dealing with wiring or internal components). \
    Respond in a friendly and helpful tone.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_input},
        {'role': 'assistant', 'content': json.dumps(product_and_issue_response, indent=4)}
    ]

    final_response = get_completion_from_messages(all_messages + messages)

    if debug:
        print("2.Generated response to user question.")
    all_messages = all_messages + messages[1:]
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("3.Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("3.Response passed moderation check.")

    return final_response, all_messages

# Testing the assistant
user_input = "Machine not cooling"
response, _ = process_user_message(user_input, [])
print(response)